# Investment Opportunities and Alignment

- how investments consistent with the Paris Agreement Long-term target look like
- the investments of today determine to a large extent the emissions of tomorrow

In [84]:
using CSV
using Statistics
using JSON
using DataFrames

## Helper functions

### Average years
This function is used to get the average of a given array of years. We use this later on.

In [236]:
function averageYears(row, years)
    # We loop over the years and get the value for each year from the row
    # Since the column name of the years are in Symbols we need to convert the String-year
    # From this new array we calculate the mean and cound it with 3 digits
    function getYear(year)
        value = row[1, Symbol(year)]
        if (typeof(value) === String) # In some cases we need to convert the data from the csv to a number
           parse(Float64, replace(value, "," => ".")) 
        else
            value
        end
    end
    round(mean(getYear, years), digits=3)
end

averageYears (generic function with 1 method)

## Mappings
Some variables have different names in the historic data set. Also, some regions might have a different name.

In [237]:
HISTORIC_VARIABLE_MAPPING = Dict{String,String}(
    "Energy Efficiency" => "Investment|Energy Efficiency",
    "Transmission and Distribution" => "Investment|Energy Supply|Electricity|Transmission and Distribution",
    "Nuclear" => "Investment|Energy Supply|Electricity|Nuclear",
    "Bioenergy" => "Investment|Energy Supply|Extraction|Biomass",
    "Solar" => "Investment|Energy Supply|Electricity|Solar",
    "Wind" => "Investment|Energy Supply|Electricity|Wind",
    "Fossil power generation" => "Investment|Energy Supply|Electricity|Fossil",
    "Coal extraction" => "Investment|Energy Supply|Extraction|Coal",
    "Storage" => "Investment|Energy Supply|Electricity|Battery storage",
    "other renewables" => "Investment|Energy Supply|Electricity|Other renewables"
)

HISTORIC_REGION_MAPPING = Dict{String,String}(
    "EU" => "EUR"
)

SCENARIO_VARIABLE_MAPPING = Dict{String,String}(
    "Energy Efficiency" => "Energy Efficiency",
    "Transmission and Distribution" => "Electricity - T&D and Storage",
    "Nuclear" => "Extraction and Conversion - Nuclear",
    "Bioenergy" => "Extraction and Conversion - Bioenergy",
    "Solar" => "Energy Supply|Electricity|Solar",
    "Wind" => "Energy Supply|Electricity|Wind",
    "Fossil power generation" => "Electricity - Fossil Fuels w/o CCS",
    "Hydrogen - Fossil" => "Hydrogen - Fossil",
    "Hydrogen - Non-fossil" => "Hydrogen - Non-fossil",
    "CCS" => "CCS",
    "Storage" => "Energy Supply|Electricity|Electricity Storage"
)


Dict{String,String} with 11 entries:
  "Bioenergy"                     => "Extraction and Conversion - Bioenergy"
  "Fossil power generation"       => "Electricity - Fossil Fuels w/o CCS"
  "Solar"                         => "Energy Supply|Electricity|Solar"
  "Transmission and Distribution" => "Electricity - T&D and Storage"
  "Hydrogen - Fossil"             => "Hydrogen - Fossil"
  "Wind"                          => "Energy Supply|Electricity|Wind"
  "Nuclear"                       => "Extraction and Conversion - Nuclear"
  "Storage"                       => "Energy Supply|Electricity|Electricity Storage"
  "Hydrogen - Non-fossil"         => "Hydrogen - Non-fossil"
  "CCS"                           => "CCS"
  "Energy Efficiency"             => "Energy Efficiency"

## Getting the historic data

In [238]:
# historicData = CSV.read("source/WEI2020-DataTables_supplytimeseries_BNEF.csv", decimal='.')

In [239]:
historicData = CSV.read("source/report_SENSES.csv"; delim=";")

unique(historicData[!,:Region])

12-element Array{String,1}:
 "R5LAM"      
 "R5ASIA"     
 "R5MAF"      
 "R5OECD90+EU"
 "R5REF"      
 "USA"        
 "CHN"        
 "IND"        
 "RUS"        
 "JPN"        
 "EUR"        
 "World"      

### Relevant historic years
We extract a average of three historic years from the data.

In [240]:
HISTORIC_YEARS = (2017, 2018, 2019)

(2017, 2018, 2019)

### Find historic data
We search for historic data by variable and region.

In [241]:
function findHistoricData(variable, region)
    row = filter(row -> row[:Variable] == variable && row[:Region] == region, historicData)
    if (nrow(row) === 0)
        println("$variable in $region was not found in historic data. Maybe variable name or region is spelled differently in the data set. Will use 0 as value.")
        return 0
    else
        return averageYears(row, HISTORIC_YEARS)
    end    
end

findHistoricData (generic function with 1 method)

In [242]:
# Test function
findHistoricData("Investment|Energy Efficiency", "CHN")

48.822

### Get historic data
We need an additional function to get the correct historic data as some variable need some extra processing.

In [243]:
function getHistoricData(variable)
    dictRegion = Dict{String, Dict}()
    
    for region in REGIONS
        # Use region mapping if region is available for that.
        _region = haskey(HISTORIC_REGION_MAPPING, region) ? HISTORIC_REGION_MAPPING[region] : region
        
        dict = Dict{String, Float64}()
        dict["Reference"] = 0

        if (haskey(HISTORIC_VARIABLE_MAPPING, variable)) # Some variables have different names in the historic data so we map the keys to the corresponding ones.
            dict["Reference"] = findHistoricData(HISTORIC_VARIABLE_MAPPING[variable], _region)
        elseif (variable === "Oil and Gas extraction") # Some variables need to be specificly calculated
            oil = findHistoricData("Investment|Energy Supply|Extraction|Oil", _region)
            gas = findHistoricData("Investment|Energy Supply|Extraction|Gas", _region)
            dict["Reference"] = oil + gas
        elseif (variable == "Hydrogen - Fossil" || variable == "Hydrogen - Non-fossil" || variable == "CCS") # Some variables are just 0
            dict["Reference"] = 0
        else # Some variables are the same as in the historic data set
            dict["Reference"] = findHistoricData(variable, _region)
        end

        dict["Reference"] = round(dict["Reference"], digits=3)
        for model in MODELS
            dict[model] = dict["Reference"]
        end
        dict["median"] = dict["Reference"] # Since we don’t have any models for the historic data, the median is the same
        dict["max"] = dict["Reference"] # The same applies for the maximum value
        
        dictRegion[region] = dict
    end

    return dictRegion
end

getHistoricData (generic function with 2 methods)

## Scenario data
We now load the scenario data set.

In [244]:
scenarioRuns = CSV.read("source/41560_2018_179_MOESM2_ESM-1.csv", decimal=',')

,Model,Region,Scenario,Variable,Unit
,String,String,String,String,String
1,AIM/CGE,World,CPol,Total energy investment,Billion US$2015/yr
2,AIM/CGE,World,CPol,Total energy investment (supply side),Billion US$2015/yr
3,AIM/CGE,World,CPol,Low carbon investment,Billion US$2015/yr
4,AIM/CGE,World,CPol,Low carbon investment (supply side),Billion US$2015/yr
5,AIM/CGE,World,CPol,Total_inv/GDP,--
6,AIM/CGE,World,CPol,LC_inv/GDP,--
7,AIM/CGE,World,CPol,Extraction and Conversion - Fossil Fuels,Billion US$2015/yr
8,AIM/CGE,World,CPol,Electricity - Fossil Fuels w/o CCS,Billion US$2015/yr
9,AIM/CGE,World,CPol,Hydrogen - Fossil,Billion US$2015/yr


In [245]:
# unique(scenarioRuns[!,:Variable])
unique(scenarioRuns[!,:Region])

10-element Array{String,1}:
 "World"      
 "CHN"        
 "EU"         
 "IND"        
 "USA"        
 "R5OECD90+EU"
 "R5REF"      
 "R5ASIA"     
 "R5MAF"      
 "R5LAM"      

## Defining valid values

In [246]:
VARIABLES = (
    "Oil and Gas extraction",
    "Coal extraction",
    "Fossil power generation",
    "Hydrogen - Fossil",
    "Nuclear",
    "Bioenergy",
    "Hydrogen - Non-fossil",
    "Solar",
    "Wind",
    "other renewables",
    "Transmission and Distribution",
    "Energy Efficiency",
    "CCS",
    "Storage"
)

MODELS = (
    "AIM/CGE",
    "IMAGE",
    "MESSAGEix-GLOBIOM",
    "POLES",
    "REMIND-MAgPIE",
    "WITCH-GLOBIOM"
)

REGIONS = (
    "World", # Julia thing: we actually need a comma if we have just one item in the tuple
    "CHN",
    "IND",
    "USA",
    "R5OECD90+EU",
    "R5REF",
    "R5ASIA",
    "R5MAF",
    "R5LAM",
    "EU"
)

SCENARIOS = (
    "1.5C",
    "NDC",
    "2C",
    "historic"
  # "CPol"
)

("1.5C", "NDC", "2C", "historic")

## Filtering runs

After defining the valid values, we filter the runs by checking if each attributes is present in the value tuples.

The `in` function checks if the first parameter is in second parameter.

In [247]:
function included(d)
    return in(d[:Model], MODELS) && in(d[:Region], REGIONS) && in(d[:Scenario], SCENARIOS)
end

datum = filter(included, scenarioRuns)

,Model,Region,Scenario,Variable,Unit
,String,String,String,String,String
1,AIM/CGE,World,NDC,Total energy investment,Billion US$2015/yr
2,AIM/CGE,World,NDC,Total energy investment (supply side),Billion US$2015/yr
3,AIM/CGE,World,NDC,Low carbon investment,Billion US$2015/yr
4,AIM/CGE,World,NDC,Low carbon investment (supply side),Billion US$2015/yr
5,AIM/CGE,World,NDC,Total_inv/GDP,--
6,AIM/CGE,World,NDC,LC_inv/GDP,--
7,AIM/CGE,World,NDC,Extraction and Conversion - Fossil Fuels,Billion US$2015/yr
8,AIM/CGE,World,NDC,Electricity - Fossil Fuels w/o CCS,Billion US$2015/yr
9,AIM/CGE,World,NDC,Hydrogen - Fossil,Billion US$2015/yr


In [248]:
unique(datum[!,:Variable])

31-element Array{String,1}:
 "Total energy investment"                                
 "Total energy investment (supply side)"                  
 "Low carbon investment"                                  
 "Low carbon investment (supply side)"                    
 "Total_inv/GDP"                                          
 "LC_inv/GDP"                                             
 "Extraction and Conversion - Fossil Fuels"               
 "Electricity - Fossil Fuels w/o CCS"                     
 "Hydrogen - Fossil"                                      
 "Electricity - Non-bio Renewables"                       
 "Hydrogen - Non-fossil"                                  
 "Extraction and Conversion - Bioenergy"                  
 "Extraction and Conversion - Nuclear"                    
 ⋮                                                        
 "Energy Supply|Electricity|Gas|w/o CCS"                  
 "Energy Supply|Electricity|Oil|w/ CCS"                   
 "Energy Supply|Electricity|

## Prepare functions

### Summing up values

Each year has it own column in the source data and we want to sum it up as one value.
Additionally, we need to parse the value and convert it from German notation to English.

In [249]:
NEAR_TERM_YEARS = (2025, 2030)
# MEDIUM_TERM_YEARS = (2025, 2030, 2035, 2040, 2045, 2050)

(2025, 2030)

To get a value in a DataFrame we use [x, y], where `x` is the row number and `y` the cell id. This would usually look something like `[1, :Region]`, but in this special case the id of the column is a `Symbol` with the name of the year. So we use `Symbol(year)`.

### GetValues
This function finds the rows in the DataFrame, gets every value and calculates the median.

First, we create an empty Dictionary with `Strings` as keys and `Float64` as numbers.
We loop over the `MODELS` and search for each row with the current model.
If we found one, we summarise the years and add the result to the Dictionary

After the loop, we get the values of the dictionary and calculate the mean value.

In [250]:
function getValues(scenario, variable)
    dictRegion = Dict{String, Dict}()

    for region in REGIONS
        dictModel = Dict{String, Float64}()

        for model in MODELS
            if (haskey(SCENARIO_VARIABLE_MAPPING, variable)) # Some variables have different names in the scenario data so we map the keys to the corresponding ones.
                dictModel[model] = findScenarioData(scenario, SCENARIO_VARIABLE_MAPPING[variable], region, model)
            elseif (variable == "Oil and Gas extraction") # Some variables need to be specificly calculated
                dictModel[model] = findScenarioData(scenario, "Extraction and Conversion - Fossil Fuels", region, model) * 0.9
            elseif (variable == "Coal extraction")
                dictModel[model] = findScenarioData(scenario, "Extraction and Conversion - Fossil Fuels", region, model) * 0.1
            elseif (variable == "other renewables")
                hydro = findScenarioData(scenario, "Energy Supply|Electricity|Hydro", region, model)
                ocean = findScenarioData(scenario, "Energy Supply|Electricity|Ocean", region, model)
                geo = findScenarioData(scenario, "Energy Supply|Electricity|Geothermal", region, model)
                dictModel[model] = hydro + ocean + geo
            else # Some variables are the same as in the historic data set
                dictModel[model] = findScenarioData(scenario, variable, region, model)
            end
            dictModel[model] = round(dictModel[model], digits=3)
        end
        
        dictModel["median"] = round(median(values(dictModel)), digits=3) # We calculate the median by simply getting the median of all values
        dictModel["max"] = maximum(values(dictModel)) # We calculate the maximum value from all values

        dictRegion[region] = dictModel
    end
    
    return dictRegion
end

getValues (generic function with 2 methods)

In [254]:
getValues("NDC", "Energy Supply|Electricity|Oil|w/ CCS", "World")

Dict{String,Float64} with 8 entries:
  "average"           => 0.0
  "POLES"             => 0.0
  "max"               => 0.728
  "REMIND-MAgPIE"     => 0.0
  "WITCH-GLOBIOM"     => 0.0
  "AIM/CGE"           => 0.728
  "IMAGE"             => 0.002
  "MESSAGEix-GLOBIOM" => 0.0

In [252]:
ENV["COLUMNS"] = 100

100

In [253]:
row = filter(row -> row[:Scenario] == "NDC" && row[:Region] == "World" && row[:Variable] == "Energy Supply|Electricity|Oil|w/ CCS", datum)

,Model,Region,Scenario,Variable,Unit,2015
,String,String,String,String,String,String
1,AIM/CGE,World,NDC,Energy Supply|Electricity|Oil|w/ CCS,Billion US$2015/yr,0
2,IMAGE,World,NDC,Energy Supply|Electricity|Oil|w/ CCS,Billion US$2015/yr,"0,002757331"
3,MESSAGEix-GLOBIOM,World,NDC,Energy Supply|Electricity|Oil|w/ CCS,Billion US$2015/yr,0
4,POLES,World,NDC,Energy Supply|Electricity|Oil|w/ CCS,Billion US$2015/yr,0
5,REMIND-MAgPIE,World,NDC,Energy Supply|Electricity|Oil|w/ CCS,Billion US$2015/yr,0
6,WITCH-GLOBIOM,World,NDC,Energy Supply|Electricity|Oil|w/ CCS,Billion US$2015/yr,0


### Filter function
The filter function takes two parameter: a function returning true or false for each item; and a list of items.

In [214]:
function findScenarioData(scenario, variable, region, model)
    row = filter(row -> row[:Scenario] == scenario && row[:Region] == region && row[:Variable] == variable && row[:Model] == model, datum)
    if (size(row, 1) > 0)
        return averageYears(row, NEAR_TERM_YEARS)
    else
        println("$variable in $region for $model and $scenario was not found in scenario data. Maybe variable name or region is spelled differently in the data set. Will use 0 as value.")
        return 0
    end
end

findScenarioData (generic function with 1 method)

In [118]:
begin
    findScenarioData("NDC", "Energy Supply|Electricity|Oil|w/ CCS", "World", "AIM/CGE")
end

1.241

### Calculating the change

Function to calculate the change between two values.
We calculate the absolute change and if its positive or negative. This makes it easier to process later.

In [221]:
function calcChange(reference, value)
    if (reference === value)
       return (0, true) 
    end
    ref = reference === 0.0 ? 1 : reference # In order to prevent the value from being Infinity, we change the value to 1 if it is 0
    diff = ref - value
    change = round(abs(diff) / ref, digits = 3)
    # change = round(abs(reference - value) / max(reference, value), digits=3)
    isPositive = ref <= value

    return (change, isPositive, abs(round(diff, digits = 3)))
end

calcChange (generic function with 1 method)

### Calculating the changes

This function iterates over all values, gets its refeference value and saves the change in the dictionary.

In [232]:
function calcChanges(vals, refs)
    dictRegion = Dict{String, Dict}()

    for region in REGIONS
        changes = Dict{String, Any}()
        ref = refs[region]["median"]
        for (key, val) in vals[region]
            changes[key] = calcChange(ref, val)
        end
        dictRegion[region] = changes
    end
    return dictRegion
end

calcChanges (generic function with 1 method)

## Build the final data set

Finally, we loop over all scenarios, variables and regions and calculate everything.

In [222]:
runs = []

for scenario in SCENARIOS
    for variable in VARIABLES
        if (scenario == "historic")
            vals = getHistoricData(variable)
            refs = vals
            changes = calcChanges(vals, refs)
        else
            vals = getValues(scenario, variable)
            refs = getHistoricData(variable)
            changes = calcChanges(vals, refs)
        end
        push!(runs, (scenario = scenario, variable = variable, values = vals, reference = refs, changes = changes))
    end
end

## Export the dataset

`JSON.json` takes two arguments: a Dict and the indent.

In [223]:
open("../static/data/investments.json", "w") do io
   write(io, JSON.json(runs, 2));
end;